In [64]:
import pandas as pd
from arcgis import GIS
gis = GIS()
import requests
import arcpy
from arcgis.features import FeatureLayer


In [82]:
# Define the URL
url = "https://services1.arcgis.com/taguadKoI1XFwivx/arcgis/rest/services/forecasts_logfile/FeatureServer/0/query"

# Base parameters for the request
params = {
    "f": "json",
    "where": "1=1",
    "outFields": "*",
    "returnGeometry": "false",  # If it's a stand-alone table, geometry isn't required.
    "outSR": "4326",  # Specify output spatial reference if needed
    "returnExceededLimitFeatures": "true"
}

all_features = []

# Pagination parameters
batch_size = 1000
offset = 0

while True:
    # Adjust parameters for pagination
    params['resultOffset'] = offset
    params['resultRecordCount'] = batch_size

    # Send the request
    response = requests.post(url, params=params)  # Use POST since some servers may have GET length limits

    # Check if the request was successful
    if response.status_code == 200:
        data = response.json()
        features = data.get('features', [])
        
        # Check if no features were returned, which means we're done
        if not features:
            break
        
        # Append features to the master list
        all_features.extend(features)
        
        # Increase the offset for next iteration
        offset += batch_size

    else:
        print("Error fetching data:", response.status_code)
        break

# Extract attribute table
attribute_table = [feature['attributes'] for feature in all_features]

# Convert to DataFrame
dfLog = pd.DataFrame(attribute_table)

#from datetime import datetime

# Generate a filename with the current timestamp in the specified folder
#timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
#filename = f"backup\\logfile_backup_{timestamp}.csv"

## Save the DataFrame to a CSV file
#df.to_csv(filename, index=False)
#print(f"Data saved to {filename}")

In [25]:
dfLog

,SEGID,EDITKEY,ADJ2023,ADJ2028,ADJ2032,ADJ2042,ADJ2050,NOTES,TIMESTAMP,ObjectId,NOTES_FURREV,TIMESTAMP_MT
0,dummy,dummy,0,0,0,0,0,dummy,2023-08-23 21:06:15.675000+00:00,1,None,2023-08-23 15:06:15.675000-06:00
1,0013_000.0,bill,0,0,-500,0,0,Smoothing,2023-08-23 15:35:00+00:00,2,None,2023-08-23 09:35:00-06:00
2,0013_000.0,bill,10000,0,-500,0,0,Smoothing,2023-08-23 15:39:38+00:00,3,None,2023-08-23 09:39:38-06:00
3,0013_000.0,bill,0,0,-500,0,0,Smoothing,2023-08-23 15:40:25+00:00,5,None,2023-08-23 09:40:25-06:00
4,0013_000.0,bill,0,0,-500,0,0,Smoothing,2023-08-23 15:41:10+00:00,6,None,2023-08-23 09:41:10-06:00
...,...,...,...,...,...,...,...,...,...,...,...,...
15041,2251_001.5,3FHDmW-$w2D4UlJO,0,0,0,0,0,none,2024-02-28 00:47:18.019000+00:00,15310,,2024-02-27 17:47:18.019000-07:00
15042,2251_001.9,3FHDmW-$w2D4UlJO,0,-900,0,-400,600,smooth,2024-02-28 00:47:33.780000+00:00,15311,,2024-02-27 17:47:33.780000-07:00
15043,2251_002.5,3FHDmW-$w2D4UlJO,0,-1000,0,0,0,none,2024-02-28 00:47:43.710000+00:00,15312,,2024-02-27 17:47:43.710000-07:00
15044,WFRC_8166,3FHDmW-$w2D4UlJO,0,0,-10,20,20,smoothing,2024-02-28 00:48:03.028000+00:00,15313,,2024-02-27 17:48:03.028000-07:00


In [83]:
# Convert 'TIMESTAMP' from Unix time (seconds) to readable date-time format
dfLog['TIMESTAMP'] = pd.to_datetime(dfLog['TIMESTAMP'], unit='ms')
dfLog['TIMESTAMP'] = pd.to_datetime(dfLog['TIMESTAMP']).dt.tz_localize('UTC')
dfLog['TIMESTAMP_MT'] = dfLog['TIMESTAMP'].dt.tz_convert('America/Denver')


# MANUALLY ADD OVERRIDES SINCE NOT IN LOG
dfLog['OV_REV']    = 1
dfLog['OV_LTPRV']  = 1
dfLog['OV_ZERO']   = 1
dfLog['OV_HIADJ']  = 1
dfLog['OV_50LT19'] = 1
dfLog['OV_503X19'] = 1
dfLog['OV_FURREV'] = 1

display(dfLog)


,SEGID,EDITKEY,ADJ2023,ADJ2028,ADJ2032,ADJ2042,ADJ2050,NOTES,TIMESTAMP,ObjectId,NOTES_FURREV,TIMESTAMP_MT,OV_REV,OV_LTPRV,OV_ZERO,OV_HIADJ,OV_50LT19,OV_503X19,OV_FURREV
0,dummy,dummy,0,0,0,0,0,dummy,2023-08-23 21:06:15.675000+00:00,1,None,2023-08-23 15:06:15.675000-06:00,1,1,1,1,1,1,1
1,0013_000.0,bill,0,0,-500,0,0,Smoothing,2023-08-23 15:35:00+00:00,2,None,2023-08-23 09:35:00-06:00,1,1,1,1,1,1,1
2,0013_000.0,bill,10000,0,-500,0,0,Smoothing,2023-08-23 15:39:38+00:00,3,None,2023-08-23 09:39:38-06:00,1,1,1,1,1,1,1
3,0013_000.0,bill,0,0,-500,0,0,Smoothing,2023-08-23 15:40:25+00:00,5,None,2023-08-23 09:40:25-06:00,1,1,1,1,1,1,1
4,0013_000.0,bill,0,0,-500,0,0,Smoothing,2023-08-23 15:41:10+00:00,6,None,2023-08-23 09:41:10-06:00,1,1,1,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15043,2251_002.5,3FHDmW-$w2D4UlJO,0,-1000,0,0,0,none,2024-02-28 00:47:43.710000+00:00,15312,,2024-02-27 17:47:43.710000-07:00,1,1,1,1,1,1,1
15044,WFRC_8166,3FHDmW-$w2D4UlJO,0,0,-10,20,20,smoothing,2024-02-28 00:48:03.028000+00:00,15313,,2024-02-27 17:48:03.028000-07:00,1,1,1,1,1,1,1
15045,WFRC_8185,3FHDmW-$w2D4UlJO,0,0,12000,5000,9000,big smoothing due to area change,2024-02-28 00:48:12.106000+00:00,15314,,2024-02-27 17:48:12.106000-07:00,1,1,1,1,1,1,1
15046,1100_001.7,3FHDmW-$w2D4UlJO,200,100,0,0,0,Smoothing,2024-02-28 02:06:08.373000+00:00,15315,,2024-02-27 19:06:08.373000-07:00,1,1,1,1,1,1,1


In [84]:

# Define the specific date you're interested in
specific_date = pd.Timestamp('2024-02-27', tz='America/Denver')

# Filter the DataFrame to only include rows where the date part of 'TIMESTAMP_MT' matches the specific date
filtered_dfLog = dfLog[(dfLog['TIMESTAMP_MT'].dt.date == specific_date.date()) & (dfLog['EDITKEY']=='3FHDmW-$w2D4UlJO')]

display(filtered_dfLog)

,SEGID,EDITKEY,ADJ2023,ADJ2028,ADJ2032,ADJ2042,ADJ2050,NOTES,TIMESTAMP,ObjectId,NOTES_FURREV,TIMESTAMP_MT,OV_REV,OV_LTPRV,OV_ZERO,OV_HIADJ,OV_50LT19,OV_503X19,OV_FURREV
14283,2060_002.5,3FHDmW-$w2D4UlJO,0,0,0,0,0,new MVC crossing to north in 2023,2024-02-27 17:40:21.088000+00:00,14441,,2024-02-27 10:40:21.088000-07:00,1,1,1,1,1,1,1
14284,0085_010.3,3FHDmW-$w2D4UlJO,0,0,0,0,0,"smoothing, big jump in 2032 and 2050 due to la...",2024-02-27 17:40:44.322000+00:00,14442,,2024-02-27 10:40:44.322000-07:00,1,1,1,1,1,1,1
14285,2060_001.9,3FHDmW-$w2D4UlJO,0,0,0,0,0,smoothing,2024-02-27 17:41:17.815000+00:00,14443,,2024-02-27 10:41:17.815000-07:00,1,1,1,1,1,1,1
14286,WFRC_8350,3FHDmW-$w2D4UlJO,0,0,0,0,0,new connectivity in 2032,2024-02-27 17:41:44.117000+00:00,14444,,2024-02-27 10:41:44.117000-07:00,1,1,1,1,1,1,1
14287,2161_003.7,3FHDmW-$w2D4UlJO,20000,20000,20000,20000,20000,Match adjacent segments,2024-02-27 17:43:33.081000+00:00,14445,,2024-02-27 10:43:33.081000-07:00,1,1,1,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15043,2251_002.5,3FHDmW-$w2D4UlJO,0,-1000,0,0,0,none,2024-02-28 00:47:43.710000+00:00,15312,,2024-02-27 17:47:43.710000-07:00,1,1,1,1,1,1,1
15044,WFRC_8166,3FHDmW-$w2D4UlJO,0,0,-10,20,20,smoothing,2024-02-28 00:48:03.028000+00:00,15313,,2024-02-27 17:48:03.028000-07:00,1,1,1,1,1,1,1
15045,WFRC_8185,3FHDmW-$w2D4UlJO,0,0,12000,5000,9000,big smoothing due to area change,2024-02-28 00:48:12.106000+00:00,15314,,2024-02-27 17:48:12.106000-07:00,1,1,1,1,1,1,1
15046,1100_001.7,3FHDmW-$w2D4UlJO,200,100,0,0,0,Smoothing,2024-02-28 02:06:08.373000+00:00,15315,,2024-02-27 19:06:08.373000-07:00,1,1,1,1,1,1,1


In [88]:
filtered_dfLog_lastRow = filtered_dfLog.groupby(['SEGID'], as_index=False).agg(ADJ2023=('ADJ2023','last'),ADJ2028=('ADJ2028','last'),ADJ2032=('ADJ2032','last'),ADJ2042=('ADJ2042','last'),ADJ2050=('ADJ2050','last'),NOTES=('NOTES','last'),NOTES_FURREV=('NOTES_FURREV','last'),OV_REV=('OV_REV','last'), OV_LTPRV=('OV_LTPRV','last'), OV_ZERO=('OV_ZERO','last'), OV_HIADJ=('OV_HIADJ','last'), OV_50LT19=('OV_50LT19','last'), OV_503X19=('OV_503X19','last'), OV_FURREV=('OV_FURREV','last'))
filtered_dfLog_lastRow


,SEGID,ADJ2023,ADJ2028,ADJ2032,ADJ2042,ADJ2050,NOTES,NOTES_FURREV,OV_REV,OV_LTPRV,OV_ZERO,OV_HIADJ,OV_50LT19,OV_503X19,OV_FURREV
0,0015_311.8,0,0,-1000,-1000,1000,New capacity in 2032.,,1,1,1,1,1,1,1
1,0015_316.9,6000,6000,6000,6000,6000,Shift to Legacy in 2023 and 2050. Shift 6000 f...,,1,1,1,1,1,1,1
2,0015_317.6,6000,6000,6000,6000,6000,Shift to Legacy in 2023 and 2050.,,1,1,1,1,1,1,1
3,0015_318.3,6000,6000,6000,6000,6000,Shift to Legacy in 2023 and 2050. Shift 6000 f...,,1,1,1,1,1,1,1
4,0015_319.5,6000,6000,6000,6000,6000,Shift to Legacy in 2023 and 2050. Shift 6000 f...,,1,1,1,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
466,WFRC_8400,0,0,0,-500,1000,new phase 1 project,,1,1,1,1,1,1,1
467,WFRC_8440,-50,0,0,0,0,none,,1,1,1,1,1,1,1
468,WFRC_8442,0,900,0,-400,300,new connectivity in 2050,,1,1,1,1,1,1,1
469,WFRC_8456,0,3550,3000,2200,2500,shifting traffic to parallel road,,1,1,1,1,1,1,1


In [89]:
filtered_dfLog_lastRow[filtered_dfLog_lastRow['SEGID']=='2251_000.0']

,SEGID,ADJ2023,ADJ2028,ADJ2032,ADJ2042,ADJ2050,NOTES,NOTES_FURREV,OV_REV,OV_LTPRV,OV_ZERO,OV_HIADJ,OV_50LT19,OV_503X19,OV_FURREV
288,2251_000.0,0,0,0,0,6000,shift due to area type change,,1,1,1,1,1,1,1


In [90]:
import os

# Path to your file geodatabase and feature class
gdb_path = 'results/ForecastsWFRCFinal.gdb'
feature_class_name = 'ForecastSegments'

# Complete path to the feature class
feature_class_path = os.path.join(gdb_path, feature_class_name)

sdfForecast = pd.DataFrame.spatial.from_featureclass(feature_class_path)

# Now df contains your data as a Pandas DataFrame
display(sdfForecast.head())


,OBJECTID,SEGID,SUBAREAID,FAC_WDAVG,FAC_SPR,FAC_FAL,FAC_SPRFAL,aadtAdjFactor,DYVOL2019,DYVOL2023,...,FL_REV,FL_LTPRV,FL_ZERO,FL_HIADJ,FL_50LT19,FL_503X19,FL_FURREV,FL_SEG,CO_NAME,SHAPE
0,1,0006_141.0,1.0,0.984,1.0276,1.0316,1.0296,-32.0,1524.0,1655.0,...,0,0,0,0,0,0,0,0,Utah,"{""paths"": [[[405824.11000000034, 4423860.33], ..."
1,2,0006_146.9,1.0,0.984,1.0276,1.0316,1.0296,-36.0,1528.0,1661.0,...,0,0,0,0,0,0,0,0,Utah,"{""paths"": [[[413442.55030000024, 4422753.7282]..."
2,3,0006_149.9,1.0,0.984,1.0276,1.0316,1.0296,857.0,1558.4,1680.1,...,0,0,0,0,0,0,0,0,Utah,"{""paths"": [[[418330.7999999998, 4422866], [418..."
3,4,0006_150.6,1.0,0.984,1.0276,1.0316,1.0296,664.0,1748.2,1867.2,...,0,0,0,0,0,0,0,0,Utah,"{""paths"": [[[419421.8803000003, 4422872.2963],..."
4,5,0006_152.6,1.0,0.984,1.0276,1.0316,1.0296,-160.0,2535.8,2634.8,...,0,0,0,0,0,0,0,0,Utah,"{""paths"": [[[422596.89969999995, 4422889.2949]..."


In [95]:
# List of fields to update
listFields = ['ADJ2023','ADJ2028','ADJ2032','ADJ2042','ADJ2050','NOTES','NOTES_FURREV','OV_REV','OV_LTPRV','OV_ZERO','OV_HIADJ','OV_50LT19','OV_503X19','OV_FURREV']  # Example field names

# Ensure SEGID is the index if it isn't already, for easier access and manipulation
sdfForecast.set_index('SEGID', inplace=True, drop=False)  # Keep SEGID as a column as well
filtered_dfLog_lastRow.set_index('SEGID', inplace=True, drop=False)

# Update the fields in sdfForecast from filtered_dfLog_lastRow for matching SEGID
for field in listFields:
    # Check if the field exists in both DataFrames to avoid KeyError
    if field in sdfForecast.columns and field in filtered_dfLog_lastRow.columns:
        # Update sdfForecast's field with filtered_dfLog_lastRow's values for matching SEGIDs
        sdfForecast.loc[sdfForecast.index.isin(filtered_dfLog_lastRow.index), field] = filtered_dfLog_lastRow[field]

# Reset index if needed
sdfForecast.reset_index(drop=True, inplace=True)
sdfForecast

,OBJECTID,SEGID,SUBAREAID,FAC_WDAVG,FAC_SPR,FAC_FAL,FAC_SPRFAL,aadtAdjFactor,DYVOL2019,DYVOL2023,...,FL_REV,FL_LTPRV,FL_ZERO,FL_HIADJ,FL_50LT19,FL_503X19,FL_FURREV,FL_SEG,CO_NAME,SHAPE
0,1,0006_141.0,1.0,0.9840,1.0276,1.0316,1.02960,-32.0,1524.0,1655.0,...,0,0,0,0,0,0,0,0,Utah,"{""paths"": [[[405824.11000000034, 4423860.33], ..."
1,2,0006_146.9,1.0,0.9840,1.0276,1.0316,1.02960,-36.0,1528.0,1661.0,...,0,0,0,0,0,0,0,0,Utah,"{""paths"": [[[413442.55030000024, 4422753.7282]..."
2,3,0006_149.9,1.0,0.9840,1.0276,1.0316,1.02960,857.0,1558.4,1680.1,...,0,0,0,0,0,0,0,0,Utah,"{""paths"": [[[418330.7999999998, 4422866], [418..."
3,4,0006_150.6,1.0,0.9840,1.0276,1.0316,1.02960,664.0,1748.2,1867.2,...,0,0,0,0,0,0,0,0,Utah,"{""paths"": [[[419421.8803000003, 4422872.2963],..."
4,5,0006_152.6,1.0,0.9840,1.0276,1.0316,1.02960,-160.0,2535.8,2634.8,...,0,0,0,0,0,0,0,0,Utah,"{""paths"": [[[422596.89969999995, 4422889.2949]..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4975,4976,WFRC_8469,1.0,0.0000,0.0000,0.0000,0.00000,0.0,0.0,0.0,...,0,0,1,0,0,0,0,0,Salt Lake,"{""paths"": [[[407213.3742000004, 4504030.9891],..."
4976,4977,WFRC_8470,1.0,0.0000,0.0000,0.0000,0.00000,0.0,0.0,0.0,...,0,0,1,0,0,0,0,0,Salt Lake,"{""paths"": [[[407723.2000000002, 4503422.753000..."
4977,4978,WFRC_8471,1.0,1.0924,1.0104,1.0243,1.01735,0.0,0.0,0.0,...,0,0,1,0,0,0,0,0,Salt Lake,"{""paths"": [[[415734.59509999957, 4481608.6051]..."
4978,4979,WFRC_8472,1.0,0.0000,0.0000,0.0000,0.00000,0.0,0.0,0.0,...,0,0,1,0,0,0,0,0,Salt Lake,"{""paths"": [[[424308.5, 4489341.199999999], [42..."


In [96]:
sdfForecast[sdfForecast['SEGID']=='2251_000.0']['OV_LTPRV']

2585    1.0
Name: OV_LTPRV, dtype: float64

In [97]:
dfFlags = pd.read_json('_site/data/flags.json', orient='records')
# add flag columns
# Loop through the rows of 'dfFlags'
for index, row in dfFlags.iterrows():
    # Get the flag name from the 'flagName' column
    flag_name = row['flagName']

    # Get the flag criteria from the 'flagCriteria' column (if needed)
    flag_criteria = row['flagCriteria']

    # Add a new column to 'dfSegs' with the flag name and set it equal to evaluated criteria, prepare criteria string with replace
    expression = "dfForecast['" + flag_name + "'] = " + flag_criteria.replace("[", "dfForecast['").replace("]", "']")
    print(expression)

    # execute expression!
    exec(expression)
    #print(result)  # Output will be 11


dfForecast['FL_REV'] = (dfForecast['NOTES']=='') & (dfForecast['NOTES_FURREV']=='')
dfForecast['FL_LTPRV'] = ((dfForecast['MF2023']+dfForecast['ADJ2023'])<(dfForecast['M2019']+dfForecast['aadtAdjFactor'])) | ((dfForecast['MF2028']+dfForecast['ADJ2028'])<(dfForecast['MF2023']+dfForecast['ADJ2023'])) | ((dfForecast['MF2032']+dfForecast['ADJ2032'])<(dfForecast['MF2028']+dfForecast['ADJ2028'])) | ((dfForecast['MF2042']+dfForecast['ADJ2042'])<(dfForecast['MF2032']+dfForecast['ADJ2032'])) | ((dfForecast['MF2050']+dfForecast['ADJ2050'])<(dfForecast['MF2042']+dfForecast['ADJ2042']))
dfForecast['FL_ZERO'] = ((dfForecast['MF2023']+dfForecast['ADJ2023'])<=0) | ((dfForecast['MF2028']+dfForecast['ADJ2028'])<=0) | ((dfForecast['MF2032']+dfForecast['ADJ2032'])<=0) | ((dfForecast['MF2042']+dfForecast['ADJ2042'])<=0) | ((dfForecast['MF2050']+dfForecast['ADJ2050'])<=0)
dfForecast['FL_HIADJ'] = (((dfForecast['aadtAdjFactor']/(dfForecast['M2019']+dfForecast['aadtAdjFactor']))>0.95) | ((dfForecast['aadtAdj

In [98]:
sdfForecast[sdfForecast['SEGID']=='2251_000.0']['NOTES']

2585    shift due to area type change
Name: NOTES, dtype: object

In [99]:
# Assuming `sdf` is your Spatial DataFrame
output_location = 'results/with-update-from-log/ForecastsWFRCFinal.gdb'
output_feature_class = 'ForecastSegments'


# Check if the geodatabase exists, if not, create it
if not arcpy.Exists(output_location):
    arcpy.CreateFileGDB_management('results/with-update-from-log', 'ForecastsWFRCFinal.gdb')

# Write the Spatial DataFrame to a new feature class in the specified file geodatabase
sdfForecast.spatial.to_featureclass(location=os.path.join(output_location, output_feature_class),sanitize_columns=False)


'e:\\GitHub\\Traffic-Volume-Forecasts\\results\\with-update-from-log\\ForecastsWFRCFinal.gdb\\ForecastSegments'